In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 10000)

In [2]:
def EMA_dataframe(pair, days):
    currency = pd.read_csv("../data/external/exchange_rates/{}_M1.csv".format(pair))
    currency.columns = ["Time", "Volume", "Open", "Close", "High", "Low"]
    
    start, end = find_interval_index(currency)

    daily_ema = np.empty(end - start, dtype="float")
    date_ema = np.empty(end - start, dtype="object")
    window_size = 24*60*days
    for i in range(start, end):
        if i - start < window_size:
            daily_ema[i - start] = currency["Close"][i - window_size + 1:i + 1].mean()
        else:
            daily_ema[i - start] = calculate_EMA(currency["Close"][i], days, daily_ema[i - start - window_size])
        date_ema[i - start] = currency["Time"][i][0:14]
            
    daily_ema = pd.DataFrame({"Time": date_ema, "EMA_{}".format(days): daily_ema})
    
    return daily_ema

def calculate_EMA(close, days, previous_ema):
    return previous_ema + (2/(1 + days))*(close - previous_ema)

def EMA_merge(days_1, days_2, currency):
    ema_1 = EMA_dataframe(currency, days_1)
    ema_2 = EMA_dataframe(currency, days_2)
    return ema_1.merge(ema_2, how="outer", on="Time")

def find_interval_index(currency):
    start = 0
    end = 0
    index = 0
    for date in currency["Time"]:
        if start == 0 and date[:4] == "2018":
            start = index
        elif end == 0 and date[:4] == "2021":
            end = index
            break;
        index += 1
    return start, end

In [3]:
def AD_Dataframe(pair):
    currency = pd.read_csv("../data/external/exchange_rates/{}_M1.csv".format(pair))
    currency.columns = ["Time", "Volume", "Open", "Close", "High", "Low"]

    start, end = find_interval_index(currency)

    ad_index = np.empty(end - start, dtype="float")
    ad_date = np.empty(end - start, dtype="object")
    window_size = 24*60

    for i in range(start, end):
        close_mean = currency["Close"][i - window_size + 1:i + 1].mean()
        high = currency["High"][i - window_size + 1:i + 1].max()
        low = currency["Low"][i - window_size + 1:i + 1].min()
        volume = currency["Volume"][i - window_size + 1:i + 1].sum()
        
        ad_index[i - start] = calculate_AD(close_mean, high, low, volume)
        ad_date[i - start] = currency["Time"][i][0:14]

    ad_index = pd.DataFrame({"Time": ad_date, "A/D Index": ad_index})
    return ad_index

def calculate_AD(close, high, low, volume):
    if high - low == 0:
        return 0
    else:
        return volume*((close - low) - (high - close))/(high - low)

In [4]:
from datetime import datetime

def convert_date(exchange):
    exchange["Time"] = pd.to_datetime(exchange["Time"], format="%Y-%m-%d %H:%M")
    


In [5]:
def RSI_dataframe(pair):
    currency = pd.read_csv("../data/external/exchange_rates/{}_M1.csv".format(pair))
    currency.columns = ["Time", "Volume", "Open", "Close", "High", "Low"]

    start, end = find_interval_index(currency)
    
    period_window = 60*24*14
    day_window = 60*24
    rsi_index = np.empty(end - start, dtype="float")
    rsi_date = np.empty(end - start, dtype="object")
    rsi_gain = np.empty(end - start, dtype="float")
    rsi_loss = np.empty(end - start, dtype="float")
    
    for i in range(start, end):
        if i - start < period_window - 1:
            gain = 0
            loss = 0
            for j in range(14):
                curr_open = currency["Open"][i - day_window*(j + 1) + 1]
                curr_close = currency["Close"][i - day_window*j]
                change = curr_close - curr_open
                if change < 0:
                    loss -= change
                else:
                    gain += change
            rsi_gain[i - start] = gain/14
            rsi_loss[i - start] = loss/14
            rsi_index[i - start] = calculate_RSI(gain, loss)
        else:
            gain = 0
            loss = 0
            for j in range(14):
                curr_open = currency["Open"][i - day_window*(j + 1) + 1]
                curr_close = currency["Close"][i - day_window*j]
                change = curr_close - curr_open
                if change < 0:
                    loss -= change
                else:
                    gain += change
            curr_gain = gain/14
            curr_loss = loss/14
            rsi_gain[i - start] = curr_gain
            rsi_loss[i - start] = curr_loss
            prv_gain = rsi_gain[i - start - period_window + 1:i - start].mean()
            prv_loss = rsi_loss[i - start - period_window + 1:i - start].mean()
            rsi_index[i - start] = calculate_RSI(curr_gain, curr_loss, prv_gain, prv_loss)
            
        rsi_date[i - start] = currency["Time"][i][0:14]
    
    rsi_index = pd.DataFrame({"Time": rsi_date, "RSI": rsi_index})
    return rsi_index
    

def calculate_RSI(curr_gain, curr_loss, prv_avg_gain=None, prv_avg_loss=None): 
    if not prv_avg_gain and not prv_avg_loss:
        avg_gain = curr_gain/14
        avg_loss = curr_loss/14
        if avg_loss == 0:
            return 100
        rs = avg_gain/avg_loss
        if rs == -1:
            return 0
        return 100 - 100/(1 + rs)
    else:
        avg_gain = (prv_avg_gain*13 + curr_gain)/14
        avg_loss = (prv_avg_loss*13 + curr_loss)/14
        if avg_loss == 0:
            return 100
        rs = avg_gain/avg_loss
        if rs == -1:
            return 0
        return 100 - 100/(1 + rs)

In [10]:
eur_usd_rsi = RSI_dataframe("EURUSD")
eur_usd_ema = EMA_merge(10, 100, "EURUSD")
eur_usd_ad = AD_Dataframe("EURUSD")
eur_usd_exchange = eur_usd_rsi.merge(eur_usd_ema, how="outer", on="Time")
eur_usd_exchange = eur_usd_exchange.merge(eur_usd_ad, how="outer", on="Time")

In [11]:
convert_date(eur_usd_exchange)

In [12]:
eur_usd_exchange

,Time,RSI,EMA_10,EMA_100,A/D Index
0,2018-01-01 22:00:00,75.049549,1.186173,1.181063,-25647.399217
1,2018-01-01 22:01:00,74.379056,1.186174,1.181063,-25487.958445
2,2018-01-01 22:02:00,74.414455,1.186176,1.181064,-25323.351338
3,2018-01-01 22:03:00,74.608222,1.186177,1.181064,-25158.488609
4,2018-01-01 22:04:00,74.677820,1.186179,1.181064,-24990.674664
...,...,...,...,...,...
1120167,2020-12-31 21:55:00,69.069328,1.191297,1.210125,40804.843368
1120168,2020-12-31 21:56:00,69.094949,1.191272,1.210125,40661.682495
1120169,2020-12-31 21:57:00,69.069489,1.191222,1.210120,40483.593276
1120170,2020-12-31 21:58:00,69.061613,1.191204,1.210107,40344.637382


In [13]:
#eur_usd_exchange.to_csv("../data/processed/exchange_rate/EURUSD_exchange.csv")